In [12]:
import joblib

In [20]:
from sklearn.base import BaseEstimator, TransformerMixin
class FeatureEngineer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self  # nothing to fit

    def transform(self, X):
        X = X.copy()
        # Average monthly charge
        X['AvgMonthlyCharge'] = X.apply(lambda row: row['TotalCharges']/row['tenure'] if row['tenure']>0 else 0, axis=1)
        # Interaction term
        X['Contract_Internet'] = X['Contract'] + '_' + X['InternetService']
        # Tenure bin
        bins = [0,12,48,72]
        labels = ['Short','Medium','Long']
        X['tenure_bin'] = pd.cut(X['tenure'], bins=bins, labels=labels)
        return X

def map_yes_no(X):
    X = X.copy()
    for col in binary_cols:
        X[col] = X[col].map({'Yes':1, 'No':0})
        X[col] = X[col].fillna(0)
    return X

binary_cols = [
    "Partner", "Dependents", "PhoneService", "MultipleLines",
    "OnlineSecurity", "OnlineBackup", "DeviceProtection",
    "TechSupport", "StreamingTV", "StreamingMovies", "PaperlessBilling"
]

In [21]:
import pandas as pd

# Example: 2 new customers
X_new = pd.DataFrame([
    {
        "SeniorCitizen": 0,
        "Partner": "Yes",
        "Dependents": "No",
        "tenure": 5,
        "PhoneService": "Yes",
        "MultipleLines": "No",
        "InternetService": "Fiber optic",
        "OnlineSecurity": "No",
        "OnlineBackup": "Yes",
        "DeviceProtection": "No",
        "TechSupport": "No",
        "StreamingTV": "Yes",
        "StreamingMovies": "No",
        "Contract": "Month-to-month",
        "PaperlessBilling": "Yes",
        "PaymentMethod": "Electronic check",
        "MonthlyCharges": 75.5,
        "TotalCharges": 377.5
    },
    {
        "SeniorCitizen": 1,
        "Partner": "No",
        "Dependents": "No",
        "tenure": 50,
        "PhoneService": "Yes",
        "MultipleLines": "Yes",
        "InternetService": "DSL",
        "OnlineSecurity": "Yes",
        "OnlineBackup": "No",
        "DeviceProtection": "Yes",
        "TechSupport": "Yes",
        "StreamingTV": "No",
        "StreamingMovies": "No",
        "Contract": "Two year",
        "PaperlessBilling": "No",
        "PaymentMethod": "Bank transfer (automatic)",
        "MonthlyCharges": 50.0,
        "TotalCharges": 2500.0
    }
])

print(X_new)


   SeniorCitizen Partner Dependents  tenure PhoneService MultipleLines  \
0              0     Yes         No       5          Yes            No   
1              1      No         No      50          Yes           Yes   

  InternetService OnlineSecurity OnlineBackup DeviceProtection TechSupport  \
0     Fiber optic             No          Yes               No          No   
1             DSL            Yes           No              Yes         Yes   

  StreamingTV StreamingMovies        Contract PaperlessBilling  \
0         Yes              No  Month-to-month              Yes   
1          No              No        Two year               No   

               PaymentMethod  MonthlyCharges  TotalCharges  
0           Electronic check            75.5         377.5  
1  Bank transfer (automatic)            50.0        2500.0  


In [22]:
# pipe = pickle.load(open('pipe.pkl','rb'))
# Load the pipeline
loaded_model = joblib.load('churn_model.pkl')

# Predict on new data
y_new_pred = loaded_model.predict(X_new)
y_new_proba = loaded_model.predict_proba(X_new)[:,1]


In [23]:
y_new_pred

array([1, 0])

In [24]:
y_new_proba

array([0.86919328, 0.04534653])